<a href="https://colab.research.google.com/github/HisakaKoji/bert-japanese/blob/master/mixi_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/halhorn/deep_dialog_tutorial.git

Cloning into 'deep_dialog_tutorial'...
remote: Enumerating objects: 132, done.
remote: Total 132 (delta 0), reused 0 (delta 0), pack-reused 132
Receiving objects: 100% (132/132), 3.51 MiB | 19.04 MiB/s, done.
Resolving deltas: 100% (57/57), done.


In [2]:
%cd deep_dialog_tutorial


/content/deep_dialog_tutorial


In [3]:
# カレントディレクトリをリポジトリ直下にするおまじない
import os
while os.getcwd().split('/')[-1] != 'deep_dialog_tutorial': os.chdir('..')
print('current dir:', os.getcwd())

current dir: /content/deep_dialog_tutorial


In [4]:
!pip install sentencepiece

     |████████████████████████████████| 1.0MB 4.4MB/s 


In [0]:


import tensorflow as tf
from deepdialog.transformer.transformer import Transformer
from deepdialog.transformer.preprocess.batch_generator import BatchGenerator



In [0]:
data_path = 'data/natsume.txt'

In [0]:
batch_generator = BatchGenerator()
batch_generator.load(data_path)

In [0]:
vocab_size = batch_generator.vocab_size

In [9]:


graph = tf.Graph()
with graph.as_default():
    transformer = Transformer(
        vocab_size=vocab_size,
        hopping_num=4,
        head_num=8,
        hidden_dim=512,
        dropout_rate=0.1,
        max_length=50,
    )
    transformer.build_graph()



W0831 05:26:24.491544 139682951456640 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0831 05:26:24.589937 139682951456640 deprecation_wrapper.py:119] From /content/deep_dialog_tutorial/deepdialog/transformer/transformer.py:56: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0831 05:26:24.617337 139682951456640 deprecation_wrapper.py:119] From /content/deep_dialog_tutorial/deepdialog/transformer/transformer.py:111: The name tf.matrix_band_part is deprecated. Please use tf.linalg.band_part instead.

W0831 05:26:25.082098 139682951456640 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/autograph/impl/api.py:255:

In [0]:
save_dir = 'tmp/learning/transformer/'
log_dir = os.path.join(save_dir, 'log')
ckpt_path = os.path.join(save_dir, 'checkpoints/model.ckpt')

os.makedirs(log_dir, exist_ok=True)

In [0]:
with graph.as_default():
    global_step = tf.train.get_or_create_global_step()
    
    learning_rate = tf.placeholder(dtype=tf.float32, name='learning_rate')
    optimizer = tf.train.AdamOptimizer(
        learning_rate=learning_rate,
        beta2=0.98,
    )
    optimize_op = optimizer.minimize(transformer.loss, global_step=global_step)

    summary_op = tf.summary.merge([
        tf.summary.scalar('train/loss', transformer.loss),
        tf.summary.scalar('train/acc', transformer.acc),
        tf.summary.scalar('train/learning_rate', learning_rate),
    ], name='train_summary')
    summary_writer = tf.summary.FileWriter(log_dir, graph)
    saver = tf.train.Saver()

In [0]:
max_step = 100000
batch_size = 128
max_learning_rate = 0.0001
warmup_step = 4000

In [0]:
def get_learning_rate(step: int) -> float:
    rate = min(step ** -0.5, step * warmup_step ** -1.5) / warmup_step ** -0.5
    return max_learning_rate * rate


In [0]:
with graph.as_default():
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    step = 0

In [0]:
with graph.as_default():
    for batch in batch_generator.get_batch(batch_size=batch_size):
        feed = {
            **batch,
            learning_rate: get_learning_rate(step + 1),
        }
        _, loss, acc, step, summary = sess.run([optimize_op, transformer.loss, transformer.acc, global_step, summary_op], feed_dict=feed)
        summary_writer.add_summary(summary, step)
        
        if step % 100 == 0:
            print(f'{step}: loss: {loss},\t acc: {acc}')
            saver.save(sess, ckpt_path, global_step=step)

0: loss: 8.36015796661377,	 acc: 0.0
